# Tracking session and revision history of notebooks

* **Difficulty level**: easy
* **Time need to lean**: 10 minutes or less
* **Key points**:
  * Magic `%sessioninfo` shows the session information of all live kernels
  * Magic `%revisions` shows the git log messages of the current notebook if it is saved in a git repository  

##  <a id="magic_sessioninfo"></a> Show session information of all kernels using magic `%sessioninfo`

### Session info of live kernels

Magic `%sessioninfo` returns the session information of SoS and all the subkernels, which for example include version of the python interpreter and name and version of all imported modules for Python kernels, and the output of function `sessionInfo()` for language R.

For example, if you have a new notebook and has not started any subkernel, the magic will only return session information for the SoS kernel:

In [1]:
%sessioninfo

SoS Version,0.21.2


After you have done some calculation in a subkernel,

In [2]:
dim(mtcars)

[1] 32 11

The output would include additional information about the subkernel

In [3]:
%sessioninfo

SoS 
 
 
 SoS Version 0.21.2 
 
 
 R 
 
 
 Kernel ir 
 
 
 Language R 
 
 
 R version 3.6.1 (2019-07-05)
Platform: x86_64-apple-darwin13.4.0 (64-bit)
Running under: macOS Mojave 10.14.3

Matrix products: default
BLAS/LAPACK: /Users/bpeng1/anaconda3/envs/sos/lib/libopenblasp-r0.3.6.dylib

locale:
[1] en_US.UTF-8/en_US.UTF-8/en_US.UTF-8/C/en_US.UTF-8/en_US.UTF-8

attached base packages:
[1] stats graphics grDevices utils datasets methods base 

loaded via a namespace (and not attached):
 [1] compiler_3.6.1 IRdisplay_0.7.0 pbdZMQ_0.3-3 tools_3.6.1 
 [5] htmltools_0.3.6 pillar_1.4.2 base64enc_0.1-3 crayon_1.3.4 
 [9] Rcpp_1.0.2 uuid_0.1-2 IRkernel_1.0.2 jsonlite_1.6 
[13] digest_0.6.20 repr_1.0.1 rlang_0.4.0 evaluate_0.14

### Additional session information

Session information is not limited to version of the kernels and libraries and might include version of particular tools used for your workflow. Because there is no general way to determine what tools have been used and retrieve their information, you will have to collection such information manually.

To add additional session information to the output of `%sessioninfo` magic, you will need to pass it to the `%sessioninfo` magic explicitly as a variable. This variable should be a dictionary, with keys being the section headers and items being the session information, which can be a string, a list of strings, a dictionary, or a list of `(key, value)` pairs. 

For example, let us find the versions of two commands `rsync` and `awk`, and revision of this document in the variable `sessioninfo`.

In [4]:
%preview -n sinfo
import time
from subprocess import check_output

sinfo = {}
sinfo['Programs'] = [
        ['rsync', check_output('rsync --version | head -1', shell=True)],
        ['awk', check_output('awk -version', shell=True)]
    ]
sinfo['Extra'] = [
        ['Date', time.strftime("%d/%m/%Y")],
        ['Revision', check_output('git rev-list --count HEAD', shell=True)],
        ['HASH', check_output('git rev-parse HEAD', shell=True)]
    ]

>>> sinfo:

{'Programs': [['rsync', b'rsync  version 2.6.9  protocol version 29\n'],
  ['awk', b'awk version 20070501\n']],
 'Extra': [['Date', '25/12/2019'],
  ['Revision', b'3\n'],
  ['HASH', b'79f0ef557b224f9286aad2a657210357945ffe39\n']]}

The information will be added automatically to the output of `%sessioninfo`.

In [5]:
%sessioninfo --with sinfo

SoS 
 
 
 SoS Version 0.21.2 
 
 
 R 
 
 
 Kernel ir 
 
 
 Language R 
 
 
 R version 3.6.1 (2019-07-05)
Platform: x86_64-apple-darwin13.4.0 (64-bit)
Running under: macOS Mojave 10.14.3

Matrix products: default
BLAS/LAPACK: /Users/bpeng1/anaconda3/envs/sos/lib/libopenblasp-r0.3.6.dylib

locale:
[1] en_US.UTF-8/en_US.UTF-8/en_US.UTF-8/C/en_US.UTF-8/en_US.UTF-8

attached base packages:
[1] stats graphics grDevices utils datasets methods base 

loaded via a namespace (and not attached):
 [1] compiler_3.6.1 IRdisplay_0.7.0 pbdZMQ_0.3-3 tools_3.6.1 
 [5] htmltools_0.3.6 pillar_1.4.2 base64enc_0.1-3 crayon_1.3.4 
 [9] Rcpp_1.0.2 uuid_0.1-2 IRkernel_1.0.2 jsonlite_1.6 
[13] digest_0.6.20 repr_1.0.1 rlang_0.4.0 evaluate_0.14 
 
 
 Programs 
 
 
 rsync rsync version 2.6.9 protocol version 29 
 
 
 awk awk version 20070501 
 
 
 Extra 
 
 
 Date 25/12/2019 
 
 
 Revision 3 
 
 
 HASH 79f0ef557b224f9286aad2a657210357945ffe39

## <a id="magic_revisions"></a> List revisions of notebook with magic `%revisions` 

We have just showed an example that uses command `git rev-parse HEAD` to append revision information to the output of `%sessioninfo`. This was an very inconvenient way to do it because SoS provides a magic `%revisions`, which displays revisions of the current document if the document is managed by git.

This magic accepts a few options. First, any options that are acceptable to [command `git log`](https://git-scm.com/docs/git-log) can be passed to this magic. The most useful ones are options to limit the commits to display such as `-n` (number of commits to output), `--since=<date>` and `--after=<date>`. For example, you can display five most recent commits with command

In [6]:
%revisions -n 5

Revision,Author,Date,Message
cdb0f6a,Bo Peng,2019-12-24,Clear all history to reduce repository size


Another useful option is the `--source` option that allows you to link the revision to specific URL (e.g. github). This option should be specified as a string with `repo`, `revision`, `path`, and `filename` interpolated as URL to the repository, revision id, and path and name of the document respectively. Here `repo` is retrieved from the output of command `git ls-remote --get-url origin` without trailing `.git` (if available). Because SoS Notebook interpolates magics automatically, you should include variables in double braces.

For example, the github URL of a particular revision of this document is `https://github.com/REPO/blob/REVISION/PATH/TO/FILENAME` so the command to link this document to its source on github would be:

In [7]:
%revisions -n 5 --source '{{repo}}/blob/{{revision}}/{{path}}'

> %revisions -n 5 --source '{repo}/blob/{revision}/{path}'

Revision,Author,Date,Message
cdb0f6a,Bo Peng,2019-12-24,Clear all history to reduce repository size


Because `github` is the most widely used git repository, we provide the aforementioned template by default if you specify option `--source` without value. That is to say, if your document is hosted in `github.com`, you can simply use

```
%revisions --source
```
to link revisions to github.

The links do not have to point to the source of the document. For example, because we always convert this document to HTML in another directory and display it on our homepage, we can link the revisions to the HTML version as follows:

In [8]:
%revisions -n 5 \
  --source '{{repo}}/blob/{{revision}}/doc/documentation/{{filename[:-6]}}.html'

> %revisions -n 5 --source '{repo}/blob/{revision}/doc/documentation/{filename[:-6]}.html'

Revision,Author,Date,Message
cdb0f6a,Bo Peng,2019-12-24,Clear all history to reduce repository size


And if you would like to display one or more links next to the revision number, you can do so with the `--links` option:

In [9]:
%revisions -n 5 --source  \
  --links HTML '{{repo}}/blob/{{revision}}/doc/documentation/{{filename[:-6]}}.html'

> %revisions -n 5 --source --links HTML '{repo}/blob/{revision}/doc/documentation/{filename[:-6]}.html'

Revision,Author,Date,Message
cdb0f6a (HTML),Bo Peng,2019-12-24,Clear all history to reduce repository size
